In [1]:
!pip install -q gradio requests
!pip install --upgrade gradio requests -q

  You can safely remove it manually.


In [ ]:
import gradio as gr
import requests
import json
from typing import Tuple, Optional

SYMBOL_API = "https://ae9299c4f2d0.ngrok-free.app"
TEXT_API = "https://b8bbb7ea43a3.ngrok-free.app"

def validate_packaging(
    excel_file,
    pdf_file,
    product_type,
    dpi,
    packaging_width,
    packaging_height,
    confidence_threshold,
    progress=gr.Progress()
):
    """
    Complete validation pipeline - runs everything automatically!
    """

    if not excel_file or not pdf_file:
        return "Please upload both Excel and PDF files.", None, None, None

    try:
        progress(0.1, desc="Converting Excel to legal rules...")

        # Convert Excel for Symbol Detection
        with open(excel_file.name, 'rb') as f:
            response = requests.post(
                f"{SYMBOL_API}/convert_excel_logo",
                files={'excel': f},
                timeout=600
            )
        if response.status_code != 200:
            return f"Symbol Excel conversion failed: {response.json().get('message', 'Unknown error')}", None, None, None

        # Convert Excel for Text Detection
        with open(excel_file.name, 'rb') as f:
            response = requests.post(
                f"{TEXT_API}/convert_excel_text",
                files={'excel': f},
                timeout=600
            )
        if response.status_code != 200:
            return f"Text Excel conversion failed: {response.json().get('message', 'Unknown error')}", None, None, None

        progress(0.2, desc="Legal rules generated.")
        progress(0.25, desc="Loading AI models...")

        # Load Symbol Detection models
        response = requests.post(f"{SYMBOL_API}/load_models", timeout=60)
        if response.status_code != 200:
            return f"Symbol model loading failed: {response.json().get('message', 'Unknown error')}", None, None, None

        # Load Text Detection models
        response = requests.post(f"{TEXT_API}/load_models", timeout=180)
        if response.status_code != 200:
            return f"Text model loading failed: {response.json().get('message', 'Unknown error')}", None, None, None

        progress(0.35, desc="Models loaded.")
        progress(0.4, desc="Detecting symbols...")

        # Detect symbols
        with open(pdf_file.name, 'rb') as f:
            files = {'pdf': f}
            data = {
                'dpi': dpi,
                'confidence_threshold': confidence_threshold
            }
            response = requests.post(
                f"{SYMBOL_API}/detect_symbols",
                files=files,
                data=data,
                timeout=180
            )
        if response.status_code != 200:
            return f"Symbol detection failed: {response.json().get('message', 'Unknown error')}", None, None, None

        detections = response.json()
        progress(0.5, desc="Validating symbols...")

        # Validate symbols
        validation_response = requests.post(
            f"{SYMBOL_API}/validate_symbols",
            json={
                'detections': detections.get('detections', []),
                'country': "Default",
                'product_metadata': {
                    'type': product_type,
                    'width_cm': packaging_width,
                    'height_cm': packaging_height
                }
            },
            timeout=300
        )
        if validation_response.status_code != 200:
            return f"Symbol validation failed: {validation_response.json().get('message', 'Unknown error')}", None, None, None

        symbol_results = {
            "status": "success",
            "detections": detections,
            "validation": validation_response.json()
        }

        # CLEAR GPU MEMORY BEFORE TEXT DETECTION
        progress(0.55, desc="Clearing GPU memory...")
        try:
            clear_response = requests.post(f"{SYMBOL_API}/clear_gpu", timeout=10)
            if clear_response.status_code == 200:
                print("[INFO] GPU memory cleared successfully")
            else:
                print("[WARN] GPU clear failed, continuing anyway")
        except Exception as e:
            print(f"[WARN] GPU clear request failed: {e}")
            pass  # Continue even if clear fails

        # Text detection
        progress(0.6, desc="Detecting text...")
        # Detect text with retry logic for SSL errors
        max_retries = 3
        retry_count = 0
        response = None

        while retry_count < max_retries:
            try:
                with open(pdf_file.name, 'rb') as f:
                    files = {'pdf': f}
                    data = {
                        'dpi': dpi,
                        'detect_panels': 'false'
                    }

                    response = requests.post(
                        f"{TEXT_API}/detect_text",
                        files=files,
                        data=data,
                        timeout=180
                    )
                break  # Success, exit retry loop

            except requests.exceptions.SSLError as e:
                retry_count += 1
                if retry_count < max_retries:
                    progress(0.6, desc=f"SSL error, retrying ({retry_count}/{max_retries})...")
                    import time
                    time.sleep(2)  # Wait 2 seconds before retry
                else:
                    return f"Text detection failed after {max_retries} retries due to SSL error. Please restart Text Detection Flask API in Colab and update the ngrok URL.", None, None, None

            except Exception as e:
                return f"Text detection failed: {str(e)}", None, None, None

        if response is None or response.status_code != 200:
            error_msg = response.json().get('message', 'Unknown error') if response else 'No response'
            return f"Text detection failed: {error_msg}", None, None, None

        ocr = response.json()

        progress(0.8, desc="Validating text...")

        validation_response = requests.post(
            f"{TEXT_API}/validate_text",
            json={
                'ocr_results': ocr.get('ocr_results', []),
                'country': "Default",
                'product_metadata': {
                    'type': product_type,
                    'width_cm': packaging_width,
                    'height_cm': packaging_height
                }
            },
            timeout=300
        )
        if validation_response.status_code != 200:
            return f"Text validation failed: {validation_response.json().get('message', 'Unknown error')}", None, None, None

        text_results = {
            "status": "success",
            "ocr": ocr,
            "validation": validation_response.json()
        }

        progress(0.9, desc="Generating report...")

        summary = format_summary(symbol_results, text_results)
        symbol_json = json.dumps(symbol_results, indent=2)
        text_json = json.dumps(text_results, indent=2)
        progress(1.0, desc="Validation complete.")

        return summary, symbol_json, text_json, None

    except Exception as e:
        return f"Error: {str(e)}", None, None, None


def format_summary(symbol_results, text_results):
    summary = "# Validation Summary\n\n"

    if symbol_results:
        symbol_val = symbol_results.get('validation', {}).get('summary', {})
        summary += "## Symbol Detection\n"
        summary += f"- Total Rules: {symbol_val.get('total_rules', 0)}\n"
        summary += f"- Passed: {symbol_val.get('passed', 0)}\n"
        summary += f"- Failed: {symbol_val.get('failed', 0)}\n"
        summary += f"- Compliance Rate: {symbol_val.get('compliance_rate', 0):.1%}\n\n"

    if text_results:
        text_val = text_results.get('validation', {}).get('summary', {})
        summary += "## Text Detection\n"
        summary += f"- Total Rules: {text_val.get('total_rules', 0)}\n"
        summary += f"- Passed: {text_val.get('passed', 0)}\n"
        summary += f"- Failed: {text_val.get('failed', 0)}\n"
        summary += f"- Compliance Rate: {text_val.get('compliance_rate', 0):.1%}\n\n"

    if symbol_results and text_results:
        s_rate = symbol_results.get('validation', {}).get('summary', {}).get('compliance_rate', 0)
        t_rate = text_results.get('validation', {}).get('summary', {}).get('compliance_rate', 0)
        overall = (s_rate + t_rate) / 2
        summary += "## Overall Status\n"
        if overall >= 0.9:
            summary += "COMPLIANT - Packaging meets requirements.\n"
        elif overall >= 0.7:
            summary += "PARTIALLY COMPLIANT - Some issues found.\n"
        else:
            summary += "NON-COMPLIANT - Multiple violations.\n"
    return summary


def create_interface():
    with gr.Blocks(title="Packaging Compliance Validator", theme=gr.themes.Monochrome()) as demo:
        gr.Markdown("""
        # DSO System
        Simple 3-Step Process:
        1. Upload your Excel file (legal requirements)
        2. Upload your packaging PDF
        3. Click "Validate" - everything runs automatically!
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## Upload Files")

                excel_file = gr.File(
                    label="Excel File (Legal Requirements)",
                    file_types=[".xlsx", ".xls"],
                    type="filepath"
                )

                pdf_file = gr.File(
                    label="Packaging PDF",
                    file_types=[".pdf"],
                    type="filepath"
                )

                gr.Markdown("## Configuration")

                product_type = gr.Dropdown(
                    choices=["5-Pack", "Single", "Multi-Pack", "Collector"],
                    value="5-Pack",
                    label="Product Type"
                )

                with gr.Accordion("Advanced Settings", open=False):
                    dpi = gr.Slider(
                        minimum=150,
                        maximum=600,
                        value=300,
                        step=50,
                        label="DPI (Image Quality)"
                    )

                    with gr.Row():
                        packaging_width = gr.Number(value=20, label="Width (cm)")
                        packaging_height = gr.Number(value=15, label="Height (cm)")

                    confidence_threshold = gr.Slider(
                        minimum=0.1,
                        maximum=0.9,
                        value=0.25,
                        step=0.05,
                        label="Detection Confidence"
                    )

                validate_btn = gr.Button("Validate Packaging", variant="primary", size="lg")

            with gr.Column(scale=2):
                gr.Markdown("## Validation Results")

                with gr.Tabs():
                    with gr.Tab("Summary"):
                        summary_output = gr.Markdown()

                    with gr.Tab("Symbol Results"):
                        symbol_output = gr.Code(language="json", label="Symbol Detection & Validation")

                    with gr.Tab("Text Results"):
                        text_output = gr.Code(language="json", label="Text Detection & Validation")

                    with gr.Tab("Visualization"):
                        viz_output = gr.Image(label="Annotated Packaging")

        validate_btn.click(
            fn=validate_packaging,
            inputs=[
                excel_file,
                pdf_file,
                product_type,
                dpi,
                packaging_width,
                packaging_height,
                confidence_threshold
            ],
            outputs=[summary_output, symbol_output, text_output, viz_output]
        )

        gr.Markdown("""
        ---
        Notes:
        - Excel must include "Logo List - 21A" and "Text Availability - 21A" sheets.
        - Higher DPI increases quality but slows processing.
        - Model loading may take longer on first run.
        """)

    return demo


demo = create_interface()
demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7860,
    debug=True
)

C:\Users\user\AppData\Local\Temp\ipykernel_25996\302662868.py:232: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="Packaging Compliance Validator", theme=gr.themes.Monochrome()) as demo:


* Running on local URL:  http://0.0.0.0:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
